In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import pandas as pd

In [48]:
from TCGAMultiOmics.multiomics import MultiOmicsData

folder_path = "/home/jonny_admin/PycharmProjects/Bioinformatics_ExternalData/tcga-assembler/LUAD"
external_data_path = "/home/jonny_admin/PycharmProjects/Bioinformatics_ExternalData/"
luad_data = MultiOmicsData(cancer_type="LUAD", tcga_data_path=folder_path, external_data_path=external_data_path,
                           modalities=[\
                                       "GE",
                                       "MIR",
                                       "LNC",
                                       # "CNV",
                                       # "SNP",
#                                        "PRO"
                                       ])



/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


PATIENTS (522, 5)
DRUGS (461, 4)
GE (576, 20472)
MIR (494, 1870)
LNC (546, 12727)
SAMPLES (599, 6)


In [80]:
from moge.utils.distance import gower_distance
pdists = gower_distance(luad_data.LNC.gene_info[["GO terms"]])

/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/distance.py:70: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


In [114]:
luad_data.LNC.gene_info['GO terms'].value_counts().sum()

162

In [102]:
pdists[~np.isnan(pdists)]

array([1., 1., 1., ..., 1., 1., 1.])

In [100]:
print(pdists.shape, np.count_nonzero(pdists >= .99))

(80981901,) 2042492


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


In [56]:
lnc_sim_adj = compute_annotation_similarity(luad_data.LNC.gene_info, "LNC")

/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/distance.py:70: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


In [145]:
luad_data.MIR.get_genes_info()

Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')


/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:438: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")


,miR family,Seed+m8,Mature sequence,Family Conservation?,MiRBase Accession
MiRBase ID,,,,,
hsa-let-7a-1,NaN,NaN,NaN,NaN,NaN
hsa-let-7a-2,let-7a-2-3p/let-7g-3p,UGUACAG,CUGUACAGCCUCCUAGCUUUCC,-1.0,MIMAT0010195
hsa-let-7a-3,NaN,NaN,NaN,NaN,NaN
hsa-let-7b,let-7-5p/98-5p,GAGGUAG,UGAGGUAGUAGGUUGUGUGGUU,2.0,MIMAT0000063
hsa-let-7c,let-7-5p/98-5p,GAGGUAG,UGAGGUAGUAGGUUGUAUGGUU,2.0,MIMAT0000064
hsa-let-7d,let-7-5p/98-5p,GAGGUAG,AGAGGUAGUAGGUUGCAUAGUU,2.0,MIMAT0000065
hsa-let-7e,let-7-5p/98-5p,GAGGUAG,UGAGGUAGGAGGUUGUAUAGUU,2.0,MIMAT0000066
hsa-let-7f-1,let-7a-3p/let-7b-3p/let-7f-1-3p/98-3p,UAUACAA,CUAUACAAUCUAUUGCCUUCCC,-1.0,MIMAT0004486
hsa-let-7f-2,let-7f-2-3p/1185-3p,UAUACAG,CUAUACAGUCUACUGUCUUUCC,-1.0,MIMAT0004487


In [163]:
luad_data.MIR.get_genes_info()["Mature sequence"].astype(str)
# luad_data.MIR.get_genes_info()["Mature sequence"].dtype

Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')


/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:438: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")


MiRBase ID
hsa-let-7a-1                            nan
hsa-let-7a-2         CUGUACAGCCUCCUAGCUUUCC
hsa-let-7a-3                            nan
hsa-let-7b           UGAGGUAGUAGGUUGUGUGGUU
hsa-let-7c           UGAGGUAGUAGGUUGUAUGGUU
hsa-let-7d           AGAGGUAGUAGGUUGCAUAGUU
hsa-let-7e           UGAGGUAGGAGGUUGUAUAGUU
hsa-let-7f-1         CUAUACAAUCUAUUGCCUUCCC
hsa-let-7f-2         CUAUACAGUCUACUGUCUUUCC
hsa-let-7g           UGAGGUAGUAGUUUGUACAGUU
hsa-let-7i           UGAGGUAGUAGUUUGUGCUGUU
hsa-mir-1-1                             nan
hsa-mir-1-2                             nan
hsa-mir-100          CAAGCUUGUAUCUAUAGGUAUG
hsa-mir-101-1                           nan
hsa-mir-101-2                           nan
hsa-mir-103a-1                          nan
hsa-mir-103a-2      AGCUUCUUUACAGUGCUGCCUUG
hsa-mir-103b-1                          nan
hsa-mir-103b-2                          nan
hsa-mir-105-1                           nan
hsa-mir-105-2                           nan
hsa-mir-106a         

In [197]:
luad_data.MIR.get_genes_info()[["Mature sequence"]].values.reshape((luad_data.MIR.get_genes_info()[["Mature sequence"]].shape[0],-1))[0].has

Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')
Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')


/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:438: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")


numpy.ndarray

In [258]:
similarity = compute_annotation_similarity(luad_data.MIR.get_genes_info()[0:100], modality="MIR")

Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')


/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:438: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")
/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/distance.py:84: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


In [305]:
similarity_df = pd.DataFrame(similarity, index=luad_data.MIR.get_genes_list()[0:100], columns=luad_data.MIR.get_genes_list()[0:100])
similarity_df

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1260a,hsa-mir-1260b,hsa-mir-1261,hsa-mir-1262,hsa-mir-1263,hsa-mir-1264,hsa-mir-1265,hsa-mir-1266
hsa-let-7a-1,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hsa-let-7a-2,0.0,0.000000,0.0,0.307692,0.307692,0.307692,0.307692,0.384615,0.423077,0.307692,...,0.346154,0.326923,0.307692,0.288462,0.326923,0.269231,0.346154,0.326923,0.307692,0.326923
hsa-let-7a-3,NaN,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hsa-let-7b,0.0,0.307692,0.0,0.000000,0.980769,0.923077,0.942308,0.307692,0.326923,0.942308,...,0.346154,0.326923,0.192308,0.211538,0.365385,0.365385,0.326923,0.326923,0.384615,0.288462
hsa-let-7c,0.0,0.307692,0.0,0.980769,0.000000,0.942308,0.961538,0.307692,0.326923,0.942308,...,0.326923,0.346154,0.211538,0.230769,0.365385,0.365385,0.346154,0.326923,0.384615,0.288462
hsa-let-7d,0.0,0.307692,0.0,0.923077,0.942308,0.000000,0.942308,0.288462,0.307692,0.923077,...,0.307692,0.326923,0.250000,0.269231,0.346154,0.346154,0.346154,0.365385,0.365385,0.269231
hsa-let-7e,0.0,0.307692,0.0,0.942308,0.961538,0.942308,0.000000,0.307692,0.326923,0.942308,...,0.288462,0.326923,0.230769,0.250000,0.346154,0.365385,0.346154,0.307692,0.365385,0.288462
hsa-let-7f-1,0.0,0.384615,0.0,0.307692,0.307692,0.288462,0.307692,0.000000,0.423077,0.307692,...,0.346154,0.307692,0.307692,0.307692,0.326923,0.269231,0.288462,0.346154,0.307692,0.307692
hsa-let-7f-2,0.0,0.423077,0.0,0.326923,0.326923,0.307692,0.326923,0.423077,0.000000,0.346154,...,0.346154,0.307692,0.307692,0.307692,0.326923,0.269231,0.307692,0.346154,0.326923,0.307692
hsa-let-7g,0.0,0.307692,0.0,0.942308,0.942308,0.923077,0.942308,0.307692,0.346154,0.000000,...,0.326923,0.346154,0.250000,0.269231,0.346154,0.384615,0.346154,0.365385,0.346154,0.269231


In [312]:
filtered = similarity_df.loc[:,:]>0.70
filtered

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1260a,hsa-mir-1260b,hsa-mir-1261,hsa-mir-1262,hsa-mir-1263,hsa-mir-1264,hsa-mir-1265,hsa-mir-1266
hsa-let-7a-1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7a-2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7a-3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7b,False,False,False,False,True,True,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7c,False,False,False,True,False,True,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7d,False,False,False,True,True,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7e,False,False,False,True,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7f-1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7f-2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hsa-let-7g,False,False,False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [316]:
np.nonzero(filtered.values)

(array([ 3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,
         6,  6,  6,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 25, 26,
        36, 36, 37, 37, 92, 93]),
 array([ 4,  5,  6,  9, 10,  3,  5,  6,  9, 10,  3,  4,  6,  9, 10,  3,  4,
         5,  9, 10, 36, 37,  3,  4,  5,  6, 10,  3,  4,  5,  6,  9, 26, 25,
         8, 37,  8, 36, 93, 92]))

In [329]:
df = pd.DataFrame(columns=["source", "target", "weight"])

In [338]:
df

,source,target,weight
-1,hsa-mir-1260b,hsa-mir-1260a,0.903846
0,hsa-let-7b,hsa-let-7c,0.980769
1,hsa-let-7b,hsa-let-7d,0.923077
2,hsa-let-7b,hsa-let-7e,0.942308
3,hsa-let-7b,hsa-let-7g,0.942308
4,hsa-let-7b,hsa-let-7i,0.961538
5,hsa-let-7c,hsa-let-7b,0.980769
6,hsa-let-7c,hsa-let-7d,0.942308
7,hsa-let-7c,hsa-let-7e,0.961538
8,hsa-let-7c,hsa-let-7g,0.942308


In [337]:
cols = filtered.index
i=0
for x,y in zip(*np.nonzero(filtered.values)):
    df.loc[i] = [cols[x], cols[y], similarity_df.loc[cols[x],cols[y]]]
    i+=1

In [276]:
sim_g = nx.from_numpy_matrix(similarity)
nx.info(sim_g)

'Name: \nType: Graph\nNumber of nodes: 100\nNumber of edges: 3075\nAverage degree:  61.5000'

In [ ]:
similarity.

In [278]:
sim_g.edges(data=True)

[(0, 2, {'weight': nan}),
 (0, 11, {'weight': nan}),
 (0, 12, {'weight': nan}),
 (0, 14, {'weight': nan}),
 (0, 15, {'weight': nan}),
 (0, 16, {'weight': nan}),
 (0, 18, {'weight': nan}),
 (0, 19, {'weight': nan}),
 (0, 20, {'weight': nan}),
 (0, 21, {'weight': nan}),
 (0, 33, {'weight': nan}),
 (0, 34, {'weight': nan}),
 (0, 35, {'weight': nan}),
 (0, 57, {'weight': nan}),
 (0, 58, {'weight': nan}),
 (0, 63, {'weight': nan}),
 (0, 64, {'weight': nan}),
 (0, 65, {'weight': nan}),
 (0, 67, {'weight': nan}),
 (0, 68, {'weight': nan}),
 (0, 69, {'weight': nan}),
 (0, 80, {'weight': nan}),
 (0, 81, {'weight': nan}),
 (0, 83, {'weight': nan}),
 (1, 3, {'weight': 0.3076923076923077}),
 (1, 4, {'weight': 0.3076923076923077}),
 (1, 5, {'weight': 0.3076923076923077}),
 (1, 6, {'weight': 0.3076923076923077}),
 (1, 7, {'weight': 0.3846153846153846}),
 (1, 8, {'weight': 0.42307692307692313}),
 (1, 9, {'weight': 0.3076923076923077}),
 (1, 10, {'weight': 0.32692307692307687}),
 (1, 13, {'weight': 0.

In [263]:
mask = np.ma.masked_where(similarity>0.7, similarity)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [265]:
mask.data[mask.mask]

array([0.98076923, 0.92307692, 0.94230769, 0.94230769, 0.96153846,
       0.98076923, 0.94230769, 0.96153846, 0.94230769, 0.94230769,
       0.92307692, 0.94230769, 0.94230769, 0.92307692, 0.92307692,
       0.94230769, 0.96153846, 0.94230769, 0.94230769, 0.92307692,
       0.84615385, 0.82692308, 0.94230769, 0.94230769, 0.92307692,
       0.94230769, 0.96153846, 0.96153846, 0.94230769, 0.92307692,
       0.92307692, 0.96153846, 1.        , 1.        , 0.84615385,
       0.98076923, 0.82692308, 0.98076923, 0.90384615, 0.90384615])

In [231]:
1-pdist/np.nanmax(pdist)

array([       nan,        nan,        nan, ..., 0.54545455, 0.5       ,
       0.81818182])

# Heterogeneous Network

In [211]:
pairwise2.align.globalxx("GCUUCUGUAGUGUAGUC", "AUCGGGCCCUCGGCGCCGG", score_only=True)

8.0

In [45]:
from moge.network.heterogeneous_network import HeterogeneousNetwork
from moge.utils.distance import compute_expression_correlations, compute_annotation_similarity
network = HeterogeneousNetwork(modalities=["MIR", "LNC", "GE"], multi_omics_data=luad_data)
network.G.number_of_edges()

In [32]:
# Adds miRNA-target interaction network
network.add_edges_from_edgelist(edgelist=luad_data.MIR.get_miRNA_target_interaction_edgelist(), 
                                modalities=["MIR", "GE"])

Adding edgelist with 284 total unique MIR genes (source), but only matching 247 nodes
Adding edgelist with 13035 total unique GE genes (target), but only matching 12234 nodes


In [33]:
# Adds Gene Regulatory Network edges
network.add_edges_from_edgelist(edgelist=luad_data.GE.get_RegNet_GRN_edgelist(),
                               modalities=["GE", "GE"])

Adding edgelist with 2072 total unique GE genes (source), but only matching 1426 nodes
Adding edgelist with 23158 total unique GE genes (target), but only matching 17694 nodes


In [34]:
# Adds miRNA-lncRNA interaction network
network.add_edges_from_edgelist(edgelist=luad_data.LNC.get_starBase_lncRNA_miRNA_interactions_edgelist(),
                               modalities=["LNC", "MIR"])
network.add_edges_from_edgelist(edgelist=luad_data.LNC.get_lncRNome_miRNA_binding_sites_edgelist(),
                               modalities=["LNC", "MIR"])

Adding edgelist with 1127 total unique LNC genes (source), but only matching 589 nodes
Adding edgelist with 275 total unique MIR genes (target), but only matching 238 nodes


In [42]:
print("isolated nodes:", len(nx.isolates(network.get_subgraph(modalities=["MIR", "GE", "LNC"]))),
      ", out of total nodes:", network.all_nodes.__len__())
print("edges: ", network.get_subgraph(modalities=["MIR", "GE", "LNC"]).number_of_edges())

isolated nodes:  15293 , out of total nodes: 35069
edges:  363895


In [76]:
print(len(network.G.nodes()))
print(len(network.G.edges()))
# print(luad_data.LNC.get_genes_list())

41338
519546


In [ ]:
mir_ge_corr = network.compute_multiomics_correlations(modalities=["MIR", "GE"])

In [47]:
# Scatter plot of original graph adjacency matrix
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
plt.imshow(compute_annotation_similarity(luad_data.LNC.get_genes_info(), "LNC"), interpolation='nearest', cmap=plt.cm.bwr)
plt.colorbar()
plt.show()

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:262: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.HGNC_lncrna_info.groupby("symbol").first(), on="symbol",


ValueError: columns overlap but no suffix specified: Index(['name', 'location', 'ensembl_gene_id'], dtype='object')

<Figure size 1080x1080 with 0 Axes>

# Laplacian Matrix

In [ ]:
import community
G = nx.read_edgelist('moge/data/karate.edgelist', create_using=nx.Graph())


In [ ]:
A = nx.adjacency_matrix(G)
A.setdiag(A.diagonal()*2)
A = A.todense()

In [ ]:
# -np.sqrt(np.sum(A, axis=1).A1)
# D = np.diag(-np.sqrt(np.sum(A, axis=1).A1))
D = np.diag(np.sum(A, axis=1).A1)

In [ ]:
L = D-A

In [ ]:
print(nx.incidence_matrix(G, oriented=True).shape)
pd.DataFrame(nx.incidence_matrix(G, oriented=True)*nx.incidence_matrix(G, oriented=True).todense().T)

In [ ]:
pd.DataFrame(nx.laplacian_matrix(G).todense())

In [ ]:
from scipy.linalg import decomp_svd
U, s, Vh = decomp_svd.svd(nx.laplacian_matrix(G).todense())

In [ ]:
U[0,:].shape

In [ ]:
partition = community.best_partition(G)
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
# Scatter plot of original graph adjacency matrix
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
plt.imshow(U, interpolation='nearest', cmap=plt.cm.bwr)
plt.colorbar()
plt.show()

# Import Graph Embedding Methods

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

from gem.utils import graph_util, plot_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as gr
from gem.evaluation.metrics import getPrecisionReport
from gem.evaluation.evaluate_link_prediction import evaluateStaticLinkPrediction

from time import time

from gem.embedding.gf       import GraphFactorization
from gem.embedding.hope     import HOPE
from gem.embedding.lap      import LaplacianEigenmaps
from gem.embedding.lle      import LocallyLinearEmbedding
from gem.embedding.node2vec import node2vec
from gem.embedding.sdne     import SDNE
from moge.embedding.dual_graph_embedding import DualGraphEmbedding


# File that contains the edges. Format: source target
# Optionally, you can add weights as third column: source target weight

# Specify whether the edges are directed
isDirected = True

# Load graph
# G = graph_util.loadGraphFromEdgeListTxt(edge_f, directed=isDirected)
# G = G.to_directed()

# G = nx.read_edgelist('../nuclei-segmentation/notebooks/gem/data/ppi.edgelist', create_using=nx.DiGraph())
G = nx.read_edgelist('moge/data/karate.edgelist', create_using=nx.DiGraph())

# G = network.G

# G = nx.convert_node_labels_to_integers(G)

In [ ]:
models = []
# You can comment out the methods you don't want to run
# models.append(HOPE(d=100, beta=0.1))
models.append(GraphFactorization(d=200, max_iter=100, eta=1*10**-4, regu=1.0))
# models.append(LaplacianEigenmaps(d=2))
# models.append(LocallyLinearEmbedding(d=100))
# models.append(node2vec(d=100, max_iter=1, walk_len=80, num_walks=10, con_size=10, ret_p=1, inout_p=1))
models.append(SDNE(d=200, beta=5, alpha=1e-5, nu1=1e-6, nu2=1e-6, 
                   K=3, n_units=[50, 15,], rho=0.3, 
                   n_iter=100, xeta=0.01, n_batch=500))

models.append(DualGraphEmbedding(d=100, reg=1.0, lr=0.05, iterations=100))
                   

In [ ]:
print ('Num nodes: %d, num edges: %d' % (G.number_of_nodes(), G.number_of_edges()))
for embedding in models:
    try:
        print("\n", embedding.get_method_name(), embedding.get_method_summary())
        t1 = time()
        
    #     Learn embedding - accepts a networkx graph or file with edge list
#         embedding.learn_embedding(graph=G, edge_f=None, is_weighted=True, no_python=True)
#         print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))

#         Y = embedding.get_embedding()

#         # Evaluate on graph reconstruction
#         MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G, embedding, Y, None)
#         print ("Graph Reconstruction. MAP:", MAP)

        
        # Evaluate on link prediction
        MAP, prec_curv = evaluateStaticLinkPrediction(G, embedding,
                                     train_ratio=0.8,
                                     is_undirected=False)
        print ("Link Prediction. MAP:", MAP)
        print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
        
        # Visualize
#         viz.plot_embedding2D(embedding.get_embedding(), di_graph=G, node_colors=None)
#         plt.show()
    except Exception as e:
        print("could not run", embedding, e)

# Plot Adjacency Matrix

In [ ]:
# Scatter plot of original graph adjacency matrix
graph_adj = nx.laplacian_matrix(G.to_undirected()).todense()
plt.figure(figsize=(9,9))
plt.imshow(graph_adj, interpolation='nearest', cmap=plt.cm.gray)
plt.colorbar()
plt.show()

In [ ]:
# Scatter plot of reconstructed adjacency matrix
plt.figure(figsize=(9,9))
plt.imshow(models[0].get_reconstructed_adj(), interpolation='nearest', cmap=plt.cm.gray)
plt.colorbar()
plt.show()

In [ ]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2)
node_pos = model.fit_transform(models[0].get_embedding())
pos = {}
for i in range(len(G.nodes())):
    pos[i] = node_pos[i, :]
nx.draw_networkx(G, pos, width=0.1, node_size=300, arrows=False,
                             alpha=0.8, font_size=12)